## Test Preprocessor

In [1]:
from src.text_processing import *

en_core_web_sm is already installed.


In [2]:
from src.general import *

In [3]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [4]:
text = read_text_file('../data/text3.txt')

In [5]:
chunked_text = add_chunk_markers(text)

## Test Planner

In [6]:
from src.planner import generate_plan

In [7]:
import uuid

# Generate a unique invocation ID for this workflow run
invocation_id = str(uuid.uuid4())



In [8]:
session_id="session_12345"

In [9]:
plan = await generate_plan(
    session_id=session_id,
    invocation_id=invocation_id,
    model="gpt-4o",
    text=chunked_text, 
    fact = 1, 
    inference = 1, 
    table_name = "plan_metadata", 
    database_file = "../database/mcq_metadata.db")

2025-10-21 09:31:09,376 - INFO - generate_plan invoked: invocation_id=266ae392-2ab3-4b13-ba23-83d09a0b1f56 model=gpt-4o fact=1 inference=1 text_len=2764
2025-10-21 09:31:09,378 - INFO - Table 'plan_metadata' already exists.
2025-10-21 09:31:09,384 - INFO - Planner agent initialized with model: gpt-4o
2025-10-21 09:31:21,666 - INFO - HTTP Request: POST https://api-main-poc.aiml.asu.edu/queryV2 "HTTP/1.1 200 OK"
2025-10-21 09:31:21,669 - INFO - Parsed plan JSON successfully (facts=1, inferences=1)
2025-10-21 09:31:21,673 - INFO - Metadata inserted into 'plan_metadata'.
2025-10-21 09:31:21,675 - INFO - Plan metadata inserted into DB: table=plan_metadata invocation_id=266ae392-2ab3-4b13-ba23-83d09a0b1f56


In [10]:
plan

{'session_id': 'session_12345',
 'system_prompt': '[ROLE]:\n  You are an experienced college instructor specializing in developing reading comprehension questions to evaluate students\' understanding of academic texts, such as textbook chapters and academic articles. You excel at identifying and selecting key facts and inferences from these texts for question development.\n\n[TASK]:\n  Your task is to summarize the academic text provided by the user and select the specified number of facts and inferences as requested by the user following the guidelines below. \n\n[Guidelines]\n  Step1: Summarize the text \n  The text is divided into chunks (as indicated by the html tags such as <chunk1></chunk1>). Summarize the content within each chunk, maintaining the chunk tags in your summary.\n\n  Step2: Identify key facts and/or inferences\n  Depending on the user’s requests, create a list of key facts and/or inferences from the academic text. The number of facts or inferences generated in this 

In [10]:
import inspect
print("generate_plan object:", generate_plan)
print("module:", getattr(generate_plan, "__module__", None))
print("signature:", inspect.signature(generate_plan))
print("source (first 200 chars):")
src = inspect.getsource(generate_plan)
print(src[:200])

generate_plan object: <function generate_plan at 0x000001BE81221990>
module: src.planner
signature: (invocation_id: str, model: str, text: str, fact: int, inference: int, table_name: str = 'plan_metadata', database_file: str = '../database/mcq_metadata.db') -> dict
source (first 200 chars):
async def generate_plan(invocation_id: str,
                  model: str,
                  text: str, 
                  fact: int, 
                  inference: int, 
                  table_name:st


## Test Controller

In [11]:
from src.controller_helper import *

In [12]:
fact =1
inference = 1
main_idea = 1

In [13]:
task_list = create_task_list(chunked_text, plan, fact, inference, main_idea)

2025-10-21 09:33:09,898 - INFO - create_task_list called (requested: facts=1, inferences=1, main_idea=1)
2025-10-21 09:33:09,899 - INFO - validate_and_parse_plan called (expected facts=1, inferences=1)
2025-10-21 09:33:09,901 - INFO - Counts — facts: expected=1 actual=1; inferences: expected=1 actual=1
2025-10-21 09:33:09,902 - INFO - Validation passed. Parsed summary_len=2087, facts=1, inferences=1
2025-10-21 09:33:09,904 - INFO - Validated plan: facts=1, inferences=1
2025-10-21 09:33:09,905 - INFO - extract_chunks called with 1 chunk label(s)
2025-10-21 09:33:09,906 - INFO - extract_chunks returning text_len=1770
2025-10-21 09:33:09,908 - INFO - extract_unlisted_chunks called; excluding 1 chunk label(s)
2025-10-21 09:33:09,909 - INFO - extract_unlisted_chunks kept 0 segment(s)
2025-10-21 09:33:09,910 - INFO - extract_chunks called with 2 chunk label(s)
2025-10-21 09:33:09,911 - INFO - extract_chunks returning text_len=2728
2025-10-21 09:33:09,912 - INFO - extract_unlisted_chunks call

In [14]:
task = task_list[0]
task.get("question_type", "").lower()

'fact'

### Test MCQ Generators

In [15]:
from src.mcq_generation import *

c:\Users\ytian126\Documents\repos\MCQ-generation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
questions = await generate_all_mcqs(session_id= session_id, task_list, invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    concurrency=30,
                                    )


TypeError: generate_all_mcqs() missing 1 required positional argument: 'invocation_id'

In [ ]:
questions = await generate_all_mcqs_quality_first(task_list, 
                                                  invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    ranking_metadata_table_name="ranking_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    candidate_num=5,
                                    concurrency=30,
                                    )

### Test the Quality-first MCQ generation

In [ ]:
ranking_metadata = await generate_mcq_quality_first(
    invocation_id=invocation_id,
    model="gpt-4o",
    task=task_list[0],
    mcq_table_name="mcq_metadata",
    ranking_table_name="ranking_metadata",
    database_file='../database/mcq_metadata.db',
    max_attempt = 3,
    attempt= 1,
    candidate_num = 5
)


## Test the Formatter

In [18]:
from src.formatter import *

In [19]:
reformatted_questions = reformat_mcq_metadata(questions)

## Test the Full Workflow

In [6]:
from src.general import *

# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [7]:
text = read_text_file('../data/text3.txt')

In [8]:
from src.workflow import *

In [ ]:
output = await question_generation_workflow(text, 
                                            fact = 1, 
                                            inference = 1, 
                                            main_idea = 1, 
                                            model = "gpt-5", 
                                            quality_first = True, 
                                            candidate_num =5,
                                            max_attempt_for_single_mcq = 3,
                                            plan_metadata_table_name = "plan_metadata",
                                            mcq_metadata_table_name = "mcq_metadata",
                                            evaluation_metadata_table_name = "evaluation_metadata",
                                            ranking_metadata_table_name = "ranking_metadata",
                                            workflow_metadata_table_name = "workflow_metadata", 
                                            database_file = "../database/mcq_metadata.db",
                                            concurrency = 10)


In [2]:
# Export the output from database to a CSV file 
from src.database_handler import export_table_to_csv

In [ ]:
export_table_to_csv(
    table_name="mcq_metadata",
    file_name="20250530_mcq_metadata.csv",
    database_file="../database/mcq_metadata.db",

)
